# 01_Cleaning and lemmatization data

## Цель

- обработать и нормализовать текст различными способами
  - привести тест к ниженему регистру
  - удалить стоп-слова
  - удалить спецсимволы 
- добавить дополнительные числовые фичи
  

## Краткое описание шагов

Для того чтобы правильно построить пайплайн для нормализации текста, мы должны правилно упорядочить шаги.
Основная проблема заключается в задаче NER, а также цифр, их желательно заменять на токены.

- Первый шаг пайплайна: 
    решить задачу NER для нашего корпуса,
    для этого можно использовать разные подходы, не буду все их описывать, но желательно, чтобы это была open source модель, и достаточно легкая
    были выбраны тестовые прдложения id 2, и на них протестированы следующие модели: Natasha, DeepPavlov, pyMorphy2, "Gherman/bert-base-NER-Russian" c HuggingFace, Stanza
    уверенным победителем оказалась Stanza

- Второй шаг пайплайны:
    вычистить ссылки и emoj если они вдруг в тексте есть
    удаление stop слова можно взять из Natasha
    cleantext

- Заменить цифры через регулярные выражения  на токены <nums>

- Третий шаг:
    нормализовать слова через pymorphy2

- Четвертый шаг:
    соединить все токены через пробел

- Сделать выборку из 5 и сравнить:
    
    
    


## Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns
import logging
from collections import Counter
import re
from pathlib import Path
import stanza
import re

## Initialization

In [2]:
! set KMP_DUPLICATE_LIB_OK=TRUE  # Windows (cmd)

In [3]:
pd.set_option("display.max_colwidth", None)

plt.style.use("classic")

In [4]:
PROJECT_NAME = "finam_text_classification"

# Get the current working directory
current_dir = Path.cwd()


# Traverse up to find "finam_text_classification" folder
project_root = current_dir
while project_root.name != PROJECT_NAME:
    if project_root.parent == project_root:  # Reached the filesystem root
        raise FileNotFoundError("The 'finam_text_classification' folder was not found.")
    project_root = project_root.parent

# Define the path to the input data relative to the project root
PTH_TO_INPUT_DATA = project_root / "data" / "raw" / "Test_data_marked.xlsx"

## Main pipeline

### 1. Чтение сырых данных

In [5]:
# Read data
try:
    # Use Path to check if the file exists
    if PTH_TO_INPUT_DATA.exists():
        # Read the Excel file
        df_raw = pd.read_excel(PTH_TO_INPUT_DATA)
        logging.info("File successfully loaded.")
    else:
        # Log an error if the file is not found
        logging.error(f"File not found: {PTH_TO_INPUT_DATA}")
except Exception as e:
    # Log any exceptions that occur
    logging.error(f"An error occurred: {e}")

In [6]:
df_raw.head(2)

,review_id,review_text
0,1,Элегантный отель в самом центре Дубай. Красивые и стильные номера. И жуткие цены) До посещения отеля рекомендуется ограбить банк или неожиданно разбогатеть)))Это пишу чтобы набрать 200 символов))))))
1,2,"Если стиль вашей поездки в Дубай-- это шоппинг в Дубай молле, созерцание фонтана и гастрономический ужин-- не пожалейте денег ""усилить"" все эти ощущения номером с видом на фонтан в отеле Armani. Это просто супер. Номера находятся на нижних этажах Burj Khalifa, поэтому вид на фонтан оттуда точно лучше, чем с улицы около, и даже лучше, чем с крыши этого замечательного здания. Лучше, чем из ресторана... В общем-- самый лучший. Я прямо будильник заводил и выходил смотреть фонтан на балкон (представления меняются в течение вечера). На балконах можно курить, что не может не радовать не расставшихся с этой привычкой. Но такие номера продают только на сайте самого отеля-- у перекупщиков типа booking.com их нет. В номере мы увидели реально работающий, удобный и функциональный ""умный дом""-- управление всей электроникой номера с iPad mini. Дизайн-- не просто привлекательный и интересный, но и очень функциональный. Сервис-- честные 5* Grand Luxe, мальчик на этаже, любой вопрос решается моментально, все очень быстро и ненавязчиво. Шоппинг: до Дубай Молла идет галерея. Ребята-сотрудники отеля помогут понести сумки и даже за вас их в отель отнесут, если не боитесь доверять ценный груз)) Смотровая площадка: билет продается без очереди прямо в отеле, но чтобы попасть на нее надо 2 раза пройти по галерее: туда и обратно, так как контроль безопасности находится со стороны Дубай молла. Ресторан-- не нашли к чему придраться, все вкусно, сервис прекрасный. Общий вывод: если опять окажемся в Дубае транзитом-- без вариантов, снова сюда."


### 2. NER TOKENS

In [7]:
# Инициализация Stanza NLP (загружается один раз)
nlp = stanza.Pipeline(lang="ru", processors="tokenize,ner", batch_size=128, use_gpu=True)


# Функция замены сущностей в тексте
def replace_entities(text):
    if not isinstance(text, str) or not text.strip():
        return text  # Возвращаем текст как есть, если он не строка или пустой

    # Обрабатываем текст с помощью Stanza
    doc = nlp(text)

    # Создаем словарь сущностей
    entity_map = {ent.text: ent.type for sent in doc.sentences for ent in sent.ents}

    # Заменяем сущности в тексте
    for entity, entity_type in entity_map.items():
        text = re.sub(rf"\b{re.escape(entity)}\b", "<" + entity_type + ">", text)

    return text


# Применяем функцию ко всей колонке
df_raw["review_text_ner"] = df_raw["review_text"].apply(replace_entities)

# time 3 min apx

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to C:\Users\baksa\stanza_resources\resources.json
INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| ner       | wikiner   |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [7]:
# Save data in interim
save_path = project_root / "data" / "interim"
save_path.mkdir(parents=True, exist_ok=True)  # Создаст директории, если их нет

# df_raw.to_parquet(save_path / "ner_data.parquet", index=False)
df_raw = pd.read_parquet(save_path / "ner_data.parquet")
df_raw.head(2)

,review_id,review_text,review_text_ner
0,1,Элегантный отель в самом центре Дубай. Красивые и стильные номера. И жуткие цены) До посещения отеля рекомендуется ограбить банк или неожиданно разбогатеть)))Это пишу чтобы набрать 200 символов)))))),Элегантный отель в самом центре <LOC>. Красивые и стильные номера. И жуткие цены) До посещения отеля рекомендуется ограбить банк или неожиданно разбогатеть)))Это пишу чтобы набрать 200 символов))))))
1,2,"Если стиль вашей поездки в Дубай-- это шоппинг в Дубай молле, созерцание фонтана и гастрономический ужин-- не пожалейте денег ""усилить"" все эти ощущения номером с видом на фонтан в отеле Armani. Это просто супер. Номера находятся на нижних этажах Burj Khalifa, поэтому вид на фонтан оттуда точно лучше, чем с улицы около, и даже лучше, чем с крыши этого замечательного здания. Лучше, чем из ресторана... В общем-- самый лучший. Я прямо будильник заводил и выходил смотреть фонтан на балкон (представления меняются в течение вечера). На балконах можно курить, что не может не радовать не расставшихся с этой привычкой. Но такие номера продают только на сайте самого отеля-- у перекупщиков типа booking.com их нет. В номере мы увидели реально работающий, удобный и функциональный ""умный дом""-- управление всей электроникой номера с iPad mini. Дизайн-- не просто привлекательный и интересный, но и очень функциональный. Сервис-- честные 5* Grand Luxe, мальчик на этаже, любой вопрос решается моментально, все очень быстро и ненавязчиво. Шоппинг: до Дубай Молла идет галерея. Ребята-сотрудники отеля помогут понести сумки и даже за вас их в отель отнесут, если не боитесь доверять ценный груз)) Смотровая площадка: билет продается без очереди прямо в отеле, но чтобы попасть на нее надо 2 раза пройти по галерее: туда и обратно, так как контроль безопасности находится со стороны Дубай молла. Ресторан-- не нашли к чему придраться, все вкусно, сервис прекрасный. Общий вывод: если опять окажемся в Дубае транзитом-- без вариантов, снова сюда.","Если стиль вашей поездки в <LOC>-- это шоппинг в <LOC> молле, созерцание фонтана и гастрономический ужин-- не пожалейте денег ""усилить"" все эти ощущения номером с видом на фонтан в отеле <ORG>. Это просто супер. Номера находятся на нижних этажах <LOC>, поэтому вид на фонтан оттуда точно лучше, чем с улицы около, и даже лучше, чем с крыши этого замечательного здания. Лучше, чем из ресторана... В общем-- самый лучший. Я прямо будильник заводил и выходил смотреть фонтан на балкон (представления меняются в течение вечера). На балконах можно курить, что не может не радовать не расставшихся с этой привычкой. Но такие номера продают только на сайте самого отеля-- у перекупщиков типа <MISC> их нет. В номере мы увидели реально работающий, удобный и функциональный ""умный дом""-- управление всей электроникой номера с <MISC>. Дизайн-- не просто привлекательный и интересный, но и очень функциональный. Сервис-- честные 5* <MISC>, мальчик на этаже, любой вопрос решается моментально, все очень быстро и ненавязчиво. Шоппинг: до <LOC> Молла идет галерея. Ребята-сотрудники отеля помогут понести сумки и даже за вас их в отель отнесут, если не боитесь доверять ценный груз)) Смотровая площадка: билет продается без очереди прямо в отеле, но чтобы попасть на нее надо 2 раза пройти по галерее: туда и обратно, так как контроль безопасности находится со стороны <LOC> молла. <LOC>-- не нашли к чему придраться, все вкусно, сервис прекрасный. Общий вывод: если опять окажемся в <LOC> транзитом-- без вариантов, снова сюда."


### 2. NER num feautes

In [11]:
nlp.processors["ner"].config

d:\Projects\finam_text_classification\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
INFO:stanza:Downloaded file to C:\Users\baksa\stanza_resources\resources.json
INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| ner       | wikiner   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


{'hidden_dim': 256,
 'char_hidden_dim': 1024,
 'word_emb_dim': 300,
 'char_emb_dim': 100,
 'num_layers': 1,
 'char_num_layers': 1,
 'pretrain_max_vocab': 250000,
 'word_dropout': 0,
 'locked_dropout': 0.0,
 'dropout': 0.5,
 'rec_dropout': 0,
 'char_rec_dropout': 0,
 'char_dropout': 0,
 'char': True,
 'charlm': True,
 'charlm_shorthand': 'ru_newswiki',
 'char_lowercase': False,
 'lowercase': True,
 'emb_finetune': False,
 'input_transform': True,
 'scheme': 'bioes',
 'sample_train': 1.0,
 'optim': 'sgd',
 'lr': 0.1,
 'min_lr': 0.0001,
 'momentum': 0,
 'lr_decay': 0.5,
 'patience': 3,
 'max_steps': 400000,
 'eval_interval': 4500,
 'batch_size': 32,
 'max_grad_norm': 5.0,
 'log_step': 20,
 'seed': 1234,
 'predict_tagset': 0,
 'train_scheme': None,
 'model_path': ['C:\\Users\\baksa\\stanza_resources\\ru\\ner\\wikiner.pt'],
 'dependencies': [{'pretrain_path': 'C:\\Users\\baksa\\stanza_resources\\ru\\pretrain\\fasttextwiki.pt',
   'forward_charlm_path': 'C:\\Users\\baksa\\stanza_resources\\r

In [12]:
# Print entity labels
nlp.processors["ner"].get_known_tags()

['LOC', 'MISC', 'ORG', 'PER']

In [13]:
df_raw["loc_count"] = df_raw["review_text_ner"].apply(lambda x: x.count("LOC"))
df_raw["org_count"] = df_raw["review_text_ner"].apply(lambda x: x.count("ORG"))
df_raw["misc_count"] = df_raw["review_text_ner"].apply(lambda x: x.count("MISC"))
df_raw["per_count"] = df_raw["review_text_ner"].apply(lambda x: x.count("PER"))

###  3. Specsymbols num feauters

In [14]:
df_raw["right_parenthesis_cnt"] = df_raw["review_text_ner"].apply(lambda x: x.count(")"))
df_raw["left_parenthesis_cnt"] = df_raw["review_text_ner"].apply(lambda x: x.count("("))
df_raw["point_cnt"] = df_raw["review_text_ner"].apply(lambda x: x.count("."))
df_raw["comma_cnt"] = df_raw["review_text_ner"].apply(lambda x: x.count("."))
df_raw["exlaim_cnt"] = df_raw["review_text_ner"].apply(lambda x: x.count("!"))

### 4. Нормализовать слова, сделать лемматизацию

In [22]:
import pandas as pd
from natasha import Segmenter, NewsEmbedding, NewsSyntaxParser, Doc

# Initialize Natasha components
segmenter = Segmenter()
embeddings = NewsEmbedding()
syntax_parser = NewsSyntaxParser(embeddings)


# Function to replace numbers with "NUM"
def replace_numbers(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.parse_syntax(syntax_parser)

    # Replace numbers
    return " ".join(["<NUM>" if token.text.isdigit() else token.text for token in doc.tokens])


# Apply function to DataFrame column
df_raw["review_text_ner_num"] = df_raw["review_text_ner"].apply(replace_numbers)

### 5. Count NUMs

In [23]:
df_raw["num_cnt"] = df_raw["review_text_ner_num"].apply(lambda x: x.count("NUM"))

### 6.Clean spec symbols

In [24]:
import string

special_symbols = set(string.punctuation) | set("«»—…№")


# Define function for cleaning text
def clean_lower_text(text):
    if not isinstance(text, str):
        return text  # Skip if it's not a string

    # Normalize entity formatting
    replacements = {
        "< LOC >": "<LOC>",
        "< ORG >": "<ORG>",
        "< NUM >": "<NUM>",
        "< PER >": "<PER>",
        "< MISC >": "<MISC>",
    }

    for old, new in replacements.items():
        text = text.replace(old, new)

    ls_allowed = {"<LOC>", "<ORG>", "<NUM>", "<PER>", "<MISC>"}  # Using a set for faster lookups

    # Tokenize words while preserving spaces
    words = text.split()

    # Filter words, keeping allowed words unchanged
    filtered_words = [
        word if word in ls_allowed else "".join(char if char not in special_symbols else " " for char in word)
        for word in words
    ]

    # Replace multiple spaces with a single space
    filtered_text = re.sub(r"\s+", " ", " ".join(filtered_words)).strip()

    return filtered_text.lower()


# Example DataFrame


# Apply function to column
df_raw["review_text_ner_num_clean"] = df_raw["review_text_ner_num"].apply(lambda x: clean_lower_text(x))

In [21]:
df_raw.head(2).T

,0,1
review_id,1,2
review_text,Элегантный отель в самом центре Дубай. Красивые и стильные номера. И жуткие цены) До посещения отеля рекомендуется ограбить банк или неожиданно разбогатеть)))Это пишу чтобы набрать 200 символов)))))),"Если стиль вашей поездки в Дубай-- это шоппинг в Дубай молле, созерцание фонтана и гастрономический ужин-- не пожалейте денег ""усилить"" все эти ощущения номером с видом на фонтан в отеле Armani. Это просто супер. Номера находятся на нижних этажах Burj Khalifa, поэтому вид на фонтан оттуда точно лучше, чем с улицы около, и даже лучше, чем с крыши этого замечательного здания. Лучше, чем из ресторана... В общем-- самый лучший. Я прямо будильник заводил и выходил смотреть фонтан на балкон (представления меняются в течение вечера). На балконах можно курить, что не может не радовать не расставшихся с этой привычкой. Но такие номера продают только на сайте самого отеля-- у перекупщиков типа booking.com их нет. В номере мы увидели реально работающий, удобный и функциональный ""умный дом""-- управление всей электроникой номера с iPad mini. Дизайн-- не просто привлекательный и интересный, но и очень функциональный. Сервис-- честные 5* Grand Luxe, мальчик на этаже, любой вопрос решается моментально, все очень быстро и ненавязчиво. Шоппинг: до Дубай Молла идет галерея. Ребята-сотрудники отеля помогут понести сумки и даже за вас их в отель отнесут, если не боитесь доверять ценный груз)) Смотровая площадка: билет продается без очереди прямо в отеле, но чтобы попасть на нее надо 2 раза пройти по галерее: туда и обратно, так как контроль безопасности находится со стороны Дубай молла. Ресторан-- не нашли к чему придраться, все вкусно, сервис прекрасный. Общий вывод: если опять окажемся в Дубае транзитом-- без вариантов, снова сюда."
review_text_ner,Элегантный отель в самом центре <LOC>. Красивые и стильные номера. И жуткие цены) До посещения отеля рекомендуется ограбить банк или неожиданно разбогатеть)))Это пишу чтобы набрать 200 символов)))))),"Если стиль вашей поездки в <LOC>-- это шоппинг в <LOC> молле, созерцание фонтана и гастрономический ужин-- не пожалейте денег ""усилить"" все эти ощущения номером с видом на фонтан в отеле <ORG>. Это просто супер. Номера находятся на нижних этажах <LOC>, поэтому вид на фонтан оттуда точно лучше, чем с улицы около, и даже лучше, чем с крыши этого замечательного здания. Лучше, чем из ресторана... В общем-- самый лучший. Я прямо будильник заводил и выходил смотреть фонтан на балкон (представления меняются в течение вечера). На балконах можно курить, что не может не радовать не расставшихся с этой привычкой. Но такие номера продают только на сайте самого отеля-- у перекупщиков типа <MISC> их нет. В номере мы увидели реально работающий, удобный и функциональный ""умный дом""-- управление всей электроникой номера с <MISC>. Дизайн-- не просто привлекательный и интересный, но и очень функциональный. Сервис-- честные 5* <MISC>, мальчик на этаже, любой вопрос решается моментально, все очень быстро и ненавязчиво. Шоппинг: до <LOC> Молла идет галерея. Ребята-сотрудники отеля помогут понести сумки и даже за вас их в отель отнесут, если не боитесь доверять ценный груз)) Смотровая площадка: билет продается без очереди прямо в отеле, но чтобы попасть на нее надо 2 раза пройти по галерее: туда и обратно, так как контроль безопасности находится со стороны <LOC> молла. <LOC>-- не нашли к чему придраться, все вкусно, сервис прекрасный. Общий вывод: если опять окажемся в <LOC> транзитом-- без вариантов, снова сюда."
loc_count,1,7
org_count,0,1
misc_count,0,3
per_count,0,0
right_parenthesis_cnt,10,3
left_parenthesis_cnt,0,1
point_cnt,2,17


### 7.Make lemattization

In [25]:
from natasha import Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, Doc

# Initialize Natasha components
segmenter = Segmenter()
morph_vocab = MorphVocab()
embeddings = NewsEmbedding()
morph_tagger = NewsMorphTagger(embeddings)


# Function for lemmatization using Natasha
def lemmatize_with_natasha(text):
    if not isinstance(text, str):
        return text  # Skip if not a string

    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)

    # Lemmatize each token
    for token in doc.tokens:
        token.lemmatize(morph_vocab)

    return " ".join([token.lemma for token in doc.tokens])


df_raw["lemmatized"] = df_raw["review_text_ner_num_clean"].apply(lemmatize_with_natasha)

In [30]:
def norm_tokens(text):
    # Normalize entity formatting
    replacements = {
        "< loc >": "<loc>",
        "< org >": "<org>",
        "< num >": "<num>",
        "< per >": "<per>",
        "< misc >": "<misc>",
    }

    for old, new in replacements.items():
        text = text.replace(old, new)

    return text


df_raw["lemmatized"] = df_raw["lemmatized"].apply(norm_tokens)

### 8. Delete stop words

In [32]:
import nltk
import string
from nltk.corpus import stopwords

# Download stopwords if not already present
nltk.download("stopwords")
russian_stopwords = set(stopwords.words("russian"))


# Function to remove stop words
def remove_stopwords(text):
    if not isinstance(text, str):
        return text  # Skip if not a string

    words = text.split()  # Tokenize by spaces
    filtered_words = [word for word in words if word.lower() not in russian_stopwords]  # Remove stopwords
    return " ".join(filtered_words)  # Reconstruct sentence


# Apply function to DataFrame column
df_raw["free_stop_words"] = df_raw["lemmatized"].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\baksa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
df_raw.head(2).T

,0,1
review_id,1,2
review_text,Элегантный отель в самом центре Дубай. Красивые и стильные номера. И жуткие цены) До посещения отеля рекомендуется ограбить банк или неожиданно разбогатеть)))Это пишу чтобы набрать 200 символов)))))),"Если стиль вашей поездки в Дубай-- это шоппинг в Дубай молле, созерцание фонтана и гастрономический ужин-- не пожалейте денег ""усилить"" все эти ощущения номером с видом на фонтан в отеле Armani. Это просто супер. Номера находятся на нижних этажах Burj Khalifa, поэтому вид на фонтан оттуда точно лучше, чем с улицы около, и даже лучше, чем с крыши этого замечательного здания. Лучше, чем из ресторана... В общем-- самый лучший. Я прямо будильник заводил и выходил смотреть фонтан на балкон (представления меняются в течение вечера). На балконах можно курить, что не может не радовать не расставшихся с этой привычкой. Но такие номера продают только на сайте самого отеля-- у перекупщиков типа booking.com их нет. В номере мы увидели реально работающий, удобный и функциональный ""умный дом""-- управление всей электроникой номера с iPad mini. Дизайн-- не просто привлекательный и интересный, но и очень функциональный. Сервис-- честные 5* Grand Luxe, мальчик на этаже, любой вопрос решается моментально, все очень быстро и ненавязчиво. Шоппинг: до Дубай Молла идет галерея. Ребята-сотрудники отеля помогут понести сумки и даже за вас их в отель отнесут, если не боитесь доверять ценный груз)) Смотровая площадка: билет продается без очереди прямо в отеле, но чтобы попасть на нее надо 2 раза пройти по галерее: туда и обратно, так как контроль безопасности находится со стороны Дубай молла. Ресторан-- не нашли к чему придраться, все вкусно, сервис прекрасный. Общий вывод: если опять окажемся в Дубае транзитом-- без вариантов, снова сюда."
review_text_ner,Элегантный отель в самом центре <LOC>. Красивые и стильные номера. И жуткие цены) До посещения отеля рекомендуется ограбить банк или неожиданно разбогатеть)))Это пишу чтобы набрать 200 символов)))))),"Если стиль вашей поездки в <LOC>-- это шоппинг в <LOC> молле, созерцание фонтана и гастрономический ужин-- не пожалейте денег ""усилить"" все эти ощущения номером с видом на фонтан в отеле <ORG>. Это просто супер. Номера находятся на нижних этажах <LOC>, поэтому вид на фонтан оттуда точно лучше, чем с улицы около, и даже лучше, чем с крыши этого замечательного здания. Лучше, чем из ресторана... В общем-- самый лучший. Я прямо будильник заводил и выходил смотреть фонтан на балкон (представления меняются в течение вечера). На балконах можно курить, что не может не радовать не расставшихся с этой привычкой. Но такие номера продают только на сайте самого отеля-- у перекупщиков типа <MISC> их нет. В номере мы увидели реально работающий, удобный и функциональный ""умный дом""-- управление всей электроникой номера с <MISC>. Дизайн-- не просто привлекательный и интересный, но и очень функциональный. Сервис-- честные 5* <MISC>, мальчик на этаже, любой вопрос решается моментально, все очень быстро и ненавязчиво. Шоппинг: до <LOC> Молла идет галерея. Ребята-сотрудники отеля помогут понести сумки и даже за вас их в отель отнесут, если не боитесь доверять ценный груз)) Смотровая площадка: билет продается без очереди прямо в отеле, но чтобы попасть на нее надо 2 раза пройти по галерее: туда и обратно, так как контроль безопасности находится со стороны <LOC> молла. <LOC>-- не нашли к чему придраться, все вкусно, сервис прекрасный. Общий вывод: если опять окажемся в <LOC> транзитом-- без вариантов, снова сюда."
loc_count,1,7
org_count,0,1
misc_count,0,3
per_count,0,0
right_parenthesis_cnt,10,3
left_parenthesis_cnt,0,1
point_cnt,2,17


### 9. Add more numerical features

In [40]:
df_raw["len_in_symbols"] = df_raw["review_text"].apply(len)
df_raw["len_in_words"] = df_raw["review_text"].apply(lambda x: len(x.split()))
df_raw["avr_len_words"] = df_raw["len_in_symbols"] / df_raw["len_in_words"]

### 10. Save all and final result

In [36]:
df_raw.rename(columns={"free_stop_words": "normalized_review_text"}, inplace=True)

In [42]:
# Save data in interim
save_path = project_root / "data" / "interim"
save_path.mkdir(parents=True, exist_ok=True)  # Создаст директории, если их нет

df_raw.to_parquet(save_path / "normalized_full_data.parquet", index=False)
# df_raw = pd.read_parquet(save_path / "normalized_data.parquet")

In [43]:
columns = [
    "review_id",
    "review_text",
    "normalized_review_text",
    "loc_count",
    "org_count",
    "misc_count",
    "per_count",
    "right_parenthesis_cnt",
    "left_parenthesis_cnt",
    "point_cnt",
    "comma_cnt",
    "exlaim_cnt",
    "num_cnt",
    "len_in_symbols",
    "len_in_words",
    "avr_len_words",
]
# Save data in interim
save_path = project_root / "data" / "interim"
save_path.mkdir(parents=True, exist_ok=True)  # Создаст директории, если их нет

df_raw[columns].to_parquet(save_path / "normalized_final_review_text.parquet", index=False)
# df_raw = pd.read_parquet(save_path / "normalized_data.parquet")

In [44]:
df_raw.head(1).T

,0
review_id,1
review_text,Элегантный отель в самом центре Дубай. Красивые и стильные номера. И жуткие цены) До посещения отеля рекомендуется ограбить банк или неожиданно разбогатеть)))Это пишу чтобы набрать 200 символов))))))
review_text_ner,Элегантный отель в самом центре <LOC>. Красивые и стильные номера. И жуткие цены) До посещения отеля рекомендуется ограбить банк или неожиданно разбогатеть)))Это пишу чтобы набрать 200 символов))))))
loc_count,1
org_count,0
misc_count,0
per_count,0
right_parenthesis_cnt,10
left_parenthesis_cnt,0
point_cnt,2
